<a href="https://colab.research.google.com/github/wileyw/DeepLearningDemos/blob/master/Quantization/Quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm


print(f"Torch: {torch.__version__}")

# Training settings
batch_size = 64
epochs = 20
lr = 3e-5
gamma = 0.7
seed = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed)

device = torch.device('cuda')

# Necessary to setup quantization
qconfig = torch.quantization.get_default_qat_qconfig('qnnpack')
torch.backends.quantized.engine = 'qnnpack'

Torch: 1.10.0+cu111


In [2]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


In [3]:
!nvidia-smi

Thu Dec 23 03:25:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses

     |████████████████████████████████| 131 kB 5.3 MB/s 
     |████████████████████████████████| 1.2 MB 39.8 MB/s 
     |████████████████████████████████| 895 kB 41.4 MB/s 
     |████████████████████████████████| 79 kB 7.6 MB/s 
     |████████████████████████████████| 8.5 MB 34.5 MB/s 
     |████████████████████████████████| 138 kB 46.2 MB/s 
     |████████████████████████████████| 127 kB 49.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


# Load Data & Augmentations

Download dogs vs cats dataset and input your kaggle key information

In [4]:
!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jinc2020
Your Kaggle Key: ··········


100%|██████████| 814M/814M [00:06<00:00, 125MB/s]



Extracting archive ./dogs-vs-cats-redux-kernels-edition/dogs-vs-cats-redux-kernels-edition.zip to ./dogs-vs-cats-redux-kernels-edition


In [5]:
!ls dogs-vs-cats-redux-kernels-edition/

sample_submission.csv  test.zip  train.zip


In [6]:
os.makedirs('data', exist_ok=True)
train_dir = 'data/train'
test_dir = 'data/test'

with zipfile.ZipFile('dogs-vs-cats-redux-kernels-edition/train.zip') as train_zip:
    train_zip.extractall('data')
    
with zipfile.ZipFile('dogs-vs-cats-redux-kernels-edition/test.zip') as test_zip:
    test_zip.extractall('data')

train_list = glob.glob(os.path.join(train_dir,'*.jpg'))
test_list = glob.glob(os.path.join(test_dir, '*.jpg'))

print(f"Train Data: {len(train_list)}")
print(f"Test Data: {len(test_list)}")

labels = [path.split('/')[-1].split('.')[0] for path in train_list]

Train Data: 25000
Test Data: 12500


In [7]:
train_list, valid_list = train_test_split(train_list, 
                                          test_size=0.2,
                                          stratify=labels,
                                          random_state=seed)



print(f"Train Data: {len(train_list)}")
print(f"Validation Data: {len(valid_list)}")
print(f"Test Data: {len(test_list)}")

Train Data: 20000
Validation Data: 5000
Test Data: 12500


In [8]:
train_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

val_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)


test_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

In [9]:
class CatsDogsDataset(Dataset):
    def __init__(self, file_list, transform=None):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        self.filelength = len(self.file_list)
        return self.filelength

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img)

        label = img_path.split("/")[-1].split(".")[0]
        label = 1 if label == "dog" else 0
        label = float(label)

        return img_transformed, label

train_data = CatsDogsDataset(train_list, transform=train_transforms)
valid_data = CatsDogsDataset(valid_list, transform=test_transforms)
test_data = CatsDogsDataset(test_list, transform=test_transforms)

train_loader = DataLoader(dataset = train_data, batch_size=batch_size, shuffle=True )
valid_loader = DataLoader(dataset = valid_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_data, batch_size=batch_size, shuffle=True)

print(len(train_data), len(train_loader))
print(len(valid_data), len(valid_loader))

20000 313
5000 79


## Load Fashion MNIST

In [5]:
tsfm = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1) )  # Grayscale to Color
])

fmnist_train_data = torchvision.datasets.FashionMNIST(root='/data', train=True, download=True, transform=tsfm)
fmnist_test_data = torchvision.datasets.FashionMNIST(root='/data', train=False, download=True, transform=tsfm)

train_loader = DataLoader(dataset = fmnist_train_data, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = fmnist_test_data, batch_size = batch_size, shuffle = True)

print(len(fmnist_train_data), len(train_loader))
print(len(fmnist_test_data), len(test_loader))

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting /data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting /data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting /data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting /data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /data/FashionMNIST/raw

60000 938
10000 157


# Load Model

In [ ]:
"""
model = torchvision.models.resnet18(pretrained=True)
# or any of these variants
model = torchvision.models.resnet34(pretrained=True)
model = torchvision.models.resnet50(pretrained=True)
model = torchvision.models.resnet101(pretrained=True)
model = torchvision.models.resnet152(pretrained=True)
"""
model = torchvision.models.resnet18(pretrained=True)

model = model.train()  # Set model to training mode.
# model = model.eval()

In [ ]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

# Setup Quantization

Quantization Implementations:

*   Fine-tune as-is & post-training static quantization
*   Quantization aware training
*   [Dynamic quantization
](https://pytorch.org/tutorials/recipes/recipes/dynamic_quantization.html)


## 1. Dynamic Quantization

In [ ]:
float_model = model

dq_model_fc = torch.quantization.quantize_dynamic(float_model, {torch.nn.Linear}, dtype=torch.qint8)
dq_model_conv2d = torch.quantization.quantize_dynamic(float_model, {torch.nn.Conv2d}, dtype=torch.qint8)
dq_model_bn = torch.quantization.quantize_dynamic(float_model, {torch.nn.BatchNorm2d}, dtype=torch.qint8)
dq_model = torch.quantization.quantize_dynamic(float_model, dtype=torch.qint8)

In [ ]:
def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (KB):', size/1e3)
    os.remove('temp.p')
    return size

f=print_size_of_model(float_model,"fp32")
print()

print('Quantize fc layer')
q1=print_size_of_model(dq_model_fc,"int8")
print("{0:.2f} times smaller".format(f/q1))
print()

print('Quantize convolution layer')
q2=print_size_of_model(dq_model_conv2d,"int8")
print("{0:.2f} times smaller".format(f/q2))
print()

print('Quantize batch norm layer')
q3=print_size_of_model(dq_model_bn,"int8")
print("{0:.2f} times smaller".format(f/q3))
print()

print('Quantize all layers')
q4=print_size_of_model(dq_model,"int8")
print("{0:.2f} times smaller".format(f/q4))
print()

model:  fp32  	 Size (KB): 46834.317

Quantize fc layer
model:  int8  	 Size (KB): 45299.081
1.03 times smaller

Quantize convolution layer
model:  int8  	 Size (KB): 46834.317
1.00 times smaller

Quantize batch norm layer
model:  int8  	 Size (KB): 46834.317
1.00 times smaller

Quantize all layers
model:  int8  	 Size (KB): 45299.081
1.03 times smaller



https://pytorch.org/docs/stable/quantization.html#torch.quantization.quantize_dynamic

"This is the simplest to apply form of quantization where the weights are quantized ahead of time but the activations are dynamically quantized during inference. This is used for situations where the model execution time is dominated by loading weights from memory rather than computing the matrix multiplications. This is true for for LSTM and Transformer type models with small batch size."

In [ ]:
# Let's try with transformers
!pip install transformers

In [ ]:
from transformers import BertModel

bert_model_float = BertModel.from_pretrained('bert-base-uncased')
bert_model_dq = torch.quantization.quantize_dynamic(bert_model_float, dtype=torch.qint8)

f=print_size_of_model(bert_model_float, "fp32")
q=print_size_of_model(bert_model_dq, "int8")
print("{0:.2f} times smaller".format(f/q))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


model:  fp32  	 Size (KB): 438007.537
model:  int8  	 Size (KB): 181488.333
2.41 times smaller


**TODO**
1. Finetune model to cat v. dog problem
2. Use training data as representative dataset for post-training static quantization on finetuned model & evaluate against finetuned model
3. Use training data to finetune Quantization Aware Training model

Create model for cat v. dog -> finetuning -> Post-training static quantization


 -> quantization aware training

 Note: https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

# Pre 2 and 3: Create model for post-training static quantization and quantization aware training

## 2. Post-training Static Quantization


In [24]:
model_fp32 = torchvision.models.resnet18(pretrained=True)
# model_fp32 = torchvision.models.mobilenet_v2(pretrained=True)
# print("Resnet18: ")
# print(model_fp32)
# model_fp32.eval()
# class CatDogFinetuneClassifier(nn.Module):
#     def __init__(self, cnn):
#         super(CatDogFinetuneClassifier, self).__init__()
#         self.cnn = cnn
#         self.fc1 = nn.Linear(in_features=512, out_features=1, bias=True)
#         # self.relu = nn.ReLU()
#         # self.fc2 = nn.Linear(in_features=256, out_features=2, bias=True)

#     def forward(self, img):
#         x = self.cnn(img).flatten(start_dim=1)
#         return self.fc1(x)

class QuantizedFinetuneClassifier(nn.Module):
    def __init__(self, classfier):
        super(CatDogFinetuneClassifier, self).__init__()
        self.quant = torch.quantization.QuantStub()
        self.classifier = classifier
        self.dequant = torch.quantization.DeQuantStub()

    def forward(self, img):
        x = self.quant(self.classifier(img).flatten(start_dim=1))
        return self.dequant(self.fc(x))

# for param in model_fp32.parameters():
#     param.requires_grad = False
model_fp32.fc = nn.Linear(in_features=512, out_features=10, bias = True)
model_fp32.train()
params_to_update = model_fp32.parameters()

# params_to_update = []
# for name, param in model_fp32.named_parameters():
#     if param.requires_grad == True:
#         params_to_update.append(param)
#         print("\t",name)

# model_ptsq_finetune = CatDogFinetuneClassifier(torch.nn.Sequential(*list(model_fp32.children())[:-1]))
# print("Trimmed resnet18 cnn:")
# print(model_ptsq_finetune.cnn)
# model_ptsq_finetune = model_ptsq_finetune.to('cuda')
# model_ptsq_finetune.train()
# for param in model_ptsq_finetune.cnn.parameters():
#     param.requires_grad = False
# model_ptsq_finetune.cnn.eval()

In [25]:
# Training settings
batch_size = 64
epochs = 10
lr = 1e-3
gamma = 0.7

# seed_everything(seed)

In [26]:
# loss function
# criterion = nn.CrossEntropyLoss()
criterion = nn.BCEWithLogitsLoss()
# optimizer
optimizer = optim.Adam(params_to_update, lr=lr)
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

model_fp32 = model_fp32.to(device)

for epoch in range(epochs):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(train_loader):
        # print(data)
        # label = label.reshape((-1, 1)).float()
        label = torch.nn.functional.one_hot(label, 10).float()
        data = data.to(device)
        label = label.to(device)

        output = model_fp32(data)
        # print(output.type())
        # print(label.type())
        # print(label)
        # print(data.shape, output.shape, label.shape)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label.argmax(dim=1)).float().mean()
        # acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_loader)
        epoch_loss += loss / len(train_loader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        # for data, label in valid_loader:
        for data, label in test_loader:
            data = data.to(device)
            label = torch.nn.functional.one_hot(label, 10).float()
            label = label.to(device)

            val_output = model_fp32(data)
            val_loss = criterion(val_output, label)
            # print(data.shape, val_output.shape, label.shape)

            acc = (val_output.argmax(dim=1) == label.argmax(dim=1)).float().mean()
            # acc = (val_output.argmax(dim=1) == label).float().mean()
            # epoch_val_accuracy += acc / len(valid_loader)
            # epoch_val_loss += val_loss / len(valid_loader)
            epoch_val_accuracy += acc / len(test_loader)
            epoch_val_loss += val_loss / len(test_loader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

  0%|          | 0/938 [00:00<?, ?it/s]

Epoch : 1 - loss : 0.0703 - acc: 0.8619 - val_loss : 0.0626 - val_acc: 0.8732



  0%|          | 0/938 [00:00<?, ?it/s]

Epoch : 2 - loss : 0.0507 - acc: 0.8997 - val_loss : 0.0525 - val_acc: 0.8984



  0%|          | 0/938 [00:00<?, ?it/s]

...

Epoch : 8 - loss : 0.0263 - acc: 0.9491 - val_loss : 0.0452 - val_acc: 0.9191



  0%|          | 0/938 [00:00<?, ?it/s]

Epoch : 9 - loss : 0.0227 - acc: 0.9562 - val_loss : 0.0445 - val_acc: 0.9202



  0%|          | 0/938 [00:00<?, ?it/s]

Epoch : 10 - loss : 0.0209 - acc: 0.9603 - val_loss : 0.0480 - val_acc: 0.9196



In [27]:
model_fp32 = model_fp32.to(torch.device('cpu'))  # move model to cpu
model_fp32.eval()
model_fp32.qconfig = torch.quantization.get_default_qconfig('fbgemm')
model_fp32_fused = torch.quantization.fuse_modules(model_fp32, [['conv1', 'bn1', 'relu']])
model_fp32_prepared = torch.quantization.prepare(model_fp32_fused)

# TODO: Run representative dataset here.
# input_fp32 = torch.randn(4, 1, 4, 4)
# model_fp32_prepared(input_fp32)
input_fp32 = test_loader
for data, label in tqdm(input_fp32):
    model_fp32_prepared(data)

model_int8 = torch.quantization.convert(model_fp32_prepared)

f=print_size_of_model(model_fp32, "fp32")
q=print_size_of_model(model_int8, "int8")
print("{0:.2f} times smaller".format(f/q))

/usr/local/lib/python3.7/dist-packages/torch/ao/quantization/observer.py:174: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."


  0%|          | 0/157 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/ao/quantization/observer.py:886: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  src_bin_begin // dst_bin_width, 0, self.dst_nbins - 1
/usr/local/lib/python3.7/dist-packages/torch/ao/quantization/observer.py:891: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  src_bin_end // dst_bin_width, 0, s

NameError: ignored

In [28]:
def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (KB):', size/1e3)
    os.remove('temp.p')
    return size

f=print_size_of_model(model_fp32, "fp32")
q=print_size_of_model(model_int8, "int8")
print("{0:.2f} times smaller".format(f/q))

model:  fp32  	 Size (KB): 44802.829
model:  int8  	 Size (KB): 11409.479
3.93 times smaller


In [ ]:
#(TODO): use test dataset to evaluate classifier

## 3. Quantization Aware Training


In [ ]:
float_model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm') 
# TODO: When writing post, explore other qconfigs (fbgemm == server inference, qnnpack == mobile, what else?)
# Also talk about symmetic vs assymetric quantization, etc.
float_model_fused = torch.quantization.fuse_modules(float_model,
    [['conv1', 'bn1', 'relu']])
float_model_prepared = torch.quantization.prepare_qat(float_model_fused)
model_int8 = torch.quantization.convert(float_model_prepared)

f=print_size_of_model(float_model,"fp32")
q=print_size_of_model(model_int8,"int8")
print("{0:.2f} times smaller".format(f/q))

/usr/local/lib/python3.7/dist-packages/torch/quantization/observer.py:124: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."
/usr/local/lib/python3.7/dist-packages/torch/quantization/observer.py:246: UserWarning: must run observer before calling calculate_qparams.                                        Returning default scale and zero point 
  Returning default scale and zero point "


model:  fp32  	 Size (KB): 178814.045
model:  int8  	 Size (KB): 46699.104
3.83 times smaller


In [ ]:
print(model_int8)

ResNet(
  (conv1): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=1.0, zero_point=0, padding=(3, 3))
  (bn1): Identity()
  (relu): Identity()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): QuantizedConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), scale=1.0, zero_point=0)
      (bn1): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=1.0, zero_point=0, padding=(1, 1))
      (bn2): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): QuantizedConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), scale=1.0, zero_point=0)
      (bn3): QuantizedBatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Quant